In [56]:
import openml
dataset = openml.datasets.get_dataset(10)
X, y, categorical_indicator, attribute_names = dataset.get_data(
    target=dataset.default_target_attribute, dataset_format="dataframe"
)
X_train , X_test, y_train, y_test = train_test_split(X, y)

In [57]:
X_train

,lymphatics,block_of_affere,bl_of_lymph_c,bl_of_lymph_s,by_pass,extravasates,regeneration_of,early_uptake_in,lym_nodes_dimin,lym_nodes_enlar,changes_in_lym,defect_in_node,changes_in_node,changes_in_stru,special_forms,dislocation_of,exclusion_of_no,no_of_nodes_in
130,deformed,yes,no,no,no,no,no,yes,1,2,oval,lacunar,lacunar,diluted,no,no,no,1
62,deformed,yes,yes,yes,yes,yes,yes,yes,1,4,round,lac_margin,lac_central,faint,vesicles,yes,yes,7
64,arched,no,no,no,no,no,no,yes,1,2,oval,lacunar,lacunar,coarse,no,yes,no,2
122,arched,yes,no,no,no,no,no,no,1,1,oval,lacunar,lac_margin,drop_like,no,no,no,1
118,arched,yes,no,no,no,no,no,yes,1,2,round,lac_margin,lac_margin,diluted,vesicles,no,yes,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,deformed,no,no,no,no,yes,no,yes,1,4,oval,lacunar,lacunar,coarse,vesicles,yes,yes,7
89,arched,yes,no,no,no,yes,no,yes,1,2,round,lacunar,lacunar,coarse,chalices,no,no,2
29,deformed,no,no,no,no,yes,no,yes,1,4,oval,lac_central,lac_central,grainy,vesicles,yes,yes,3
75,deformed,yes,yes,no,no,yes,no,no,1,3,oval,lac_margin,lac_margin,coarse,vesicles,no,yes,2


In [60]:
y_train

130      metastases
62     malign_lymph
64       metastases
122      metastases
118      metastases
           ...     
95     malign_lymph
89       metastases
29     malign_lymph
75       metastases
93     malign_lymph
Name: class, Length: 111, dtype: category
Categories (4, object): ['normal' < 'metastases' < 'malign_lymph' < 'fibrosis']

In [67]:
from sklearn.preprocessing import LabelBinarizer, LabelEncoder

# write preprocessor because pipeline doesn't work well with newer versions of sklearn
class PreProcessor:
  def __init__(self, steps=[]):
    self.steps = steps
    
  def fit_transform(self, X):
    ret = X.copy()
    for tf, cols in self.steps:
      to_trans = [k for k in X.keys() if k in cols]
      for k in to_trans:
        ret[k] = tf.fit_transform(X[k])
    return ret

In [69]:
from sklearn.model_selection import train_test_split

bin_cols = ['block_of_affere', 'bl_of_lymph_c', 'bl_of_lymph_s', 'by_pass', 
           'extravasates', 'regeneration_of', 'early_uptake_in', 'dislocation_of', 'exclusion_of_no']
lb_cols = ["lymphatics", "changes_in_lym", "defect_in_node", "changes_in_node", "changes_in_stru", "special_forms"]

pp = PreProcessor([(LabelBinarizer(), bin_cols), (LabelEncoder(), lb_cols)])
X_train = pp.fit_transform(X_train)
y_train = LabelEncoder().fit_transform(y_train)

In [76]:
from sklearn.svm import SVC

clf = SVC()
clf.fit(X_train, y_train)
print(clf.score(pp.fit_transform(X_test), LabelEncoder().fit_transform(y_test)))

0.7567567567567568
